<h1>human organ classification </h1>
<h2> C-NN model<h3> [transfer learn on MobileNetV2 model]</h3></h2>

call to packages & libraries

In [9]:

from tensorflow import keras
from datetime import datetime

from src.sup.evaluation import *
from src.sup.support import *
from src.sup.test_set_eval import *

from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model

In [3]:
model_name = "mobileNet_v2_transfer_learn"

# Load the TensorBoard notebook extension.
%load_ext tensorboard

# call inline plt.

In [4]:
# Clear any logs from previous runs
!rm -rf ./logs/

'rm' is not recognized as an internal or external command,
operable program or batch file.


callout dataset

In [ ]:
classes = ['heart','brain','eye','kidney','skull','other']
root_dir = '../../datasets/'
train_dir = os.path.join(root_dir,'train')
validation_dir = os.path.join(root_dir,'validation')
tr_heart_dir,tr_brain_dir,tr_eye_dir,tr_kidney_dir,tr_skull_dir = path_update(train_dir,classes)
vl_heart_dir,vl_brain_dir,vl_eye_dir,vl_kidney_dir,vl_skull_dir = path_update(validation_dir,classes)

take a glance at training dataset

In [ ]:
plot_sample_of_img(4,4,os.listdir(tr_heart_dir)+os.listdir(tr_eye_dir))

ImageGenator - autolabelling, and categorizing.

In [ ]:
train_gen_tmp = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_gen_tmp = ImageDataGenerator(rescale=1/225.)

train_gen = train_gen_tmp.flow_from_directory(train_dir,
                                              target_size=(300,300),
                                              color_mode='rgb',
                                              class_mode='categorical',
                                              batch_size= 100,
                                              shuffle=True,
                                              seed=42)

validation_gen = validation_gen_tmp.flow_from_directory(validation_dir,
                                              target_size=(300,300),
                                              color_mode='rgb',
                                              class_mode='categorical',
                                              batch_size= 100,
                                              shuffle=True,
                                              seed=42)

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=validation_gen.n//validation_gen.batch_size

clToInt_dict = train_gen.class_indices
clToInt_dict = dict((v,k) for v,k in clToInt_dict.items())

define the model

In [10]:
weight_path = '../../h5_files/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224.h5'

pre_model = MobileNetV2(input_shape=(224,224,3),
                    weights = None,
                    include_top = True)

pre_model.load_weights(weight_path)
#model.summary()

for layer in pre_model.layers:
    layer.trainable = False

conn_layer = pre_model.get_layer('block_12_add')
conn_output = conn_layer.output

x = Conv2D(256,(3,3),activation='relu')(conn_output)
x = MaxPool2D(2,2)(x)
x = Flatten()(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128,activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(6,activation='softmax')(x)

model = Model(pre_model.input,x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

compile the model

In [ ]:
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['categorical_crossentropy'])

save the log

In [ ]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

fit & train the model.

In [ ]:
history = model.fit(train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,
                    verbose=1)

visualize layer process in cnn

In [ ]:
#visualize_model(model,img_path)

instant evaluation

In [ ]:
#call to the tensorboard
%tensorboard --logdir logs
#look at training model performance
acc_n_loss(history)

model.evaluate_generator(validation_gen,
                         steps=STEP_SIZE_VALID)

evaluate the model on test set.

In [ ]:
y_pred,y_test = test_eval(model,classes)
plot_confusion_metrix(y_test,y_pred,classes)
ROC_classes(6,y_test,y_pred,classes)



save the model in .h5 file

In [ ]:
model_path,model_weight_path = save(model,datetime.now()+model_name)

make prediction on random images

In [ ]:
#rnd_predict(model_path,model_weight_path,img_path,clToInt_dict)
